In [ ]:
import pygame
import time
import math
from queue import PriorityQueue
pygame.font.init()

win = pygame.display.set_mode((540, 600))
pygame.display.set_caption("Draw")

board = [[' ' for i in range(20)] for j in range(20)]
board[0][0] = 'O'
board[19][19] = 'X'


class Grid:
    
    board = [[' ' for i in range(20)] for j in range(20)]
    
    def __init__(self, rows, cols, width, height):
        self.rows = rows
        self.cols = cols 
        self.cubes = [[Cube(self.board[i][j], i, j, width, height) for j in range(cols)] for i in range(rows)]
        self.width = width
        self.height = height
        self.model = None
        self.selected = None
        self.start = (0, 0)
        
        
    def update_model(self):
        self.model = [[self.cubes[i][j].value for j in range(self.cols)] for i in range(self.rows)]
        
    def draw(self, val):
        gap = self.width / 20
        
        thick = 1
        
        for i in range(self.rows + 1):
            pygame.draw.line(win, (0,0,0), (0, i*gap), (self.width, i*gap), thick)
            pygame.draw.line(win, (0,0,0), (i*gap, 0), (i*gap, self.height), thick)
            
            
        for i in range(self.rows):
            for j in range(self.cols):
                self.cubes[i][j].draw(win)
                
                
    def click(self, pos):
        
        if pos[0] < self.width and pos[1] < self.height:
            gap = self.height / 20
            x = pos[0]//gap
            y = pos[1]//gap
            return (int(y), int(x))
        else:
            return None
        
    def select(self, row, col):
        self.cubes[row][col].selected = True
        self.selected = (row, col)
        
    def valid(self, pos, q, c):
        for i in [-1, 0, 1]:
            for j in [-1, 0, 1]:
                if (i,j) != (0,0):
                    if (0 <= pos[0] + i <20) and (0 <= pos[1] + j<20) and (board[pos[0] + i][pos[1] + j] != '#'):
                        if (self.cubes[pos[0] + i][pos[1] + j].h, (pos[0] + i, pos[1] + j)) not in q.queue and ((pos[0] + i, pos[1] + j) not in c):
                            self.cubes[pos[0] + i][pos[1] + j].upback(self.cubes[pos[0]][pos[1]])
                            self.cubes[pos[0] + i][pos[1] + j].update(self.cubes[pos[0]][pos[1]])
                            q.put((self.cubes[pos[0] + i][pos[1] + j].h, (pos[0] + i, pos[1] + j)))
                            self.cubes[pos[0] + i][pos[1] + j].test = True
                            
                            

    def solve(self, start, q):
        x = 50
        if q.empty:
            self.valid(start, q)
    
        while q.not_empty:
            self.valid(q.get(), q)
            
            if x < 0:
                break
            x -= 1
        
    
    
class Cube:
    rows = 20
    cols = 20
    
    def __init__(self, board, row, col, width, height):
        self.row = row
        self.col = col
        self.board = board
        self.width = width
        self.height = height
        self.selected = False
        self.g = 0
        self.h = None
        self.f = None
        self.old = None
        self.next = None
        self.test = False
        self.red = False
        
    def draw(self, win):
        fnt = pygame.font.SysFont("comicsans", 40)
        
        gap = self.width / 20
        x = self.col * gap
        y = self.row * gap
        
        if self.col == 0 and self.row == 0:
            text = fnt.render('O', 1, (0,0,0))
            win.blit(text, (x+(gap/2 - text.get_width()/2), y + (gap/2 - text.get_width()/2)))
            
        elif self.col == 19 and self.row == 19:
            text = fnt.render('X', 1, (0,0,0))
            win.blit(text, (x+(gap/2 - text.get_width()/2), y + (gap/2 - text.get_width()/2)))
            
        elif self.selected:
            pygame.draw.rect(win,(0, 0, 0), (x, y, gap, gap))
            board[self.row][self.col] = '#'
            
        if self.test:
            pygame.draw.rect(win,(0, 255, 130), (x, y, gap, gap))
            
        if self.red:
            pygame.draw.rect(win,(255, 0, 0), (x, y, gap, gap))
        
            
    def upback(self, obj):
        self.old = obj.pos()
    
    
    def pos(self):
        return (self.row, self.col)
    
    def update(self, obj):
        self.g = obj.g + int(math.sqrt((self.col-self.old[0])**2 + (self.row-self.old[1])**2)*10)
        print(obj.g, self.g, int(math.sqrt((self.col-self.old[0])**2 + (self.row-self.old[1])**2)*10), self.col, self.row)
        self.h = int(math.sqrt((19-self.col)**2 + (19-self.row)**2)*10)
        print(self.h)
        self.f = int(self.g + self.h)
                
                
def redraw_window(win, board, time, strikes):
    win.fill((255,255,255))
    fnt = pygame.font.SysFont("comicsans", 40)
    board.draw(win)
    
def finished(obj, pos):
    obj[pos[0]][pos[1]].red = True
    print(pos)
                
def main():
    
    board = Grid(20, 20, 540, 540)
    key = None
    run = True
    start = time.time()
    strikes = 0
    z = False
    start_alg = False
    q = PriorityQueue()
    c = []
    p = (19,19)
    cur = (0,0)
    x = True
    while run:
        if start_alg:
            if q.not_empty and x:
                while cur in c:
                    tmp = q.get()
                    cur = tmp[1]
                board.valid(cur, q, c)
                c.append(cur)
                #print("old " ,board.cubes[cur[0]][cur[1]].g)
                #print(cur)

                if cur == (18, 18):
                    x = False

            #print(q.queue)
        if board.cubes[p[0]][p[1]].old != None:
            #print(q.queue)
            #print(c)
            finished(board.cubes, p)
            p = board.cubes[p[0]][p[1]].old
                
        
        play_time = round(time.time() - start)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    board.valid(cur, q, c)
                    c.append(cur)
                    start_alg = True
         
        if event.type == pygame.MOUSEBUTTONDOWN:
            if z == True:
                z = False
            else:
                z = True
            
        if z:
            pos = pygame.mouse.get_pos()
            clicked = board.click(pos)
            board.select(clicked[0], clicked[1])
        
        
        redraw_window(win, board, play_time, strikes)
        pygame.display.update()
        
main()
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
0 10 10 1 0
261
0 10 10 0 1
261
0 14 14 1 1
254
14 28 14 2 0
254
14 24 10 2 1
247
14 28 14 0 2
254
14 24 10 1 2
247
14 28 14 2 2
240
28 42 14 1 3
240
28 38 10 2 3
233
38 66 28 1 4
234
38 60 22 2 4
226
60 102 42 1 5
228
60 96 36 2 5
220
60 91 31 3 5
212
91 113 22 4 5
205
91 127 36 3 6
206
91 122 31 4 6
198
122 136 14 5 5
197
122 144 22 5 6
191
122 164 42 3 7
200
122 158 36 4 7
192
122 153 31 5 7
184
153 195 42 4 8
186
153 189 36 5 8
178
153 184 31 6 8
170
184 198 14 7 7
169
184 206 22 7 8
162
184 220 36 6 9
164
184 215 31 7 9
156
215 237 22 8 9
148
215 251 36 7 10
150
237 257 20 9 10
134
257 257 0 10 9
134
257 285 28 8 11
136
257 279 22 9 11
128
279 321 42 8 12
130
279 315 36 9 12
122
315 365 50 9 13
116
315 359 44 10 13
108
359 387 28 11 12
106
359 395 36 11 13
100
359 409 50 10 14
102
395 417 22 12 13
92
395 426 31 12 14
86
426 440 14 13 13
84
426 448 22 13 14
78
426 468 42 11 15
89
426 462 36 12 15
8

In [5]:
board = Grid(20, 20, 540, 540)

In [8]:
for i in range(20):
    for j in range(20):
        print(board[i][j],  end = ",")
        
    print("")

O, , , , , , , , , , , , , , , , , , , ,
 , , , , , , , , , , , , , , , , , , , ,
 , , , , , , , , , ,#, , , , , , , , , ,
 , , , , , , , , , , , , , , , , , , , ,
 , , , , , , , , , , , , , , , , , , , ,
 , , , , , , , ,#, , , , , , , , , , , ,
 , , , , , ,#,#, , , , , , , , , , , , ,
 , , , , , ,#, , , , , , , , , , , , , ,
 , , , , ,#, , , , , , , , , , , , , , ,
 , , , , ,#, , , , , , , , , , , , , , ,
 , , , , ,#, , , , , , , , , , , , , , ,
 , , , , ,#, , , , , , , , , , , , , , ,
 , , , , ,#, , , , , , , , , , , , , , ,
 , , , , , ,#,#,#, , , , , , , , , , , ,
 , , , , , , , ,#, , , , , , , , , , , ,
 , , , , , , , , ,#, , , , , , , , , , ,
 , , , , , , , , ,#, , , , , , , , , , ,
 , , , , , , , , ,#, , , , , , , , , , ,
 , , , , , , , , , , , , , , , , , , , ,
 , , , , , , , , , , , , , , , , , , ,X,


In [3]:
board[0]

['O',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ']

In [13]:
from queue import PriorityQueue

q = PriorityQueue()



In [ ]:
import queue

def printMaze(maze, path=""):
    for x, pos in enumerate(maze[0]):
        if pos == "O":
            start = x

    i = start
    j = 0
    pos = set()
    for move in path:
        if move == "L":
            i -= 1

        elif move == "R":
            i += 1

        elif move == "U":
            j -= 1

        elif move == "D":
            j += 1
        pos.add((j, i))
    
    for j, row in enumerate(maze):
        for i, col in enumerate(row):
            if (j, i) in pos:
                print("+ ", end="")
            else:
                print(col + " ", end="")
        print()
        


def valid(maze, moves):
    for x, pos in enumerate(maze[0]):
        if pos == "O":
            start = x

    i = start
    j = 0
    for move in moves:
        if move == "L":
            i -= 1

        elif move == "R":
            i += 1

        elif move == "U":
            j -= 1

        elif move == "D":
            j += 1

        if not(0 <= i < len(maze[0]) and 0 <= j < len(maze)):
            return False
        elif (maze[j][i] == "#"):
            return False

    return True


def findEnd(maze, moves):
    for x, pos in enumerate(maze[0]):
        if pos == "O":
            start = x

    i = start
    j = 0
    for move in moves:
        if move == "L":
            i -= 1

        elif move == "R":
            i += 1

        elif move == "U":
            j -= 1

        elif move == "D":
            j += 1

    if maze[j][i] == "X":
        print("Found: " + moves)
        printMaze(maze, moves)
        return True

    return False


# MAIN ALGORITHM

nums = queue.Queue()
nums.put("")
add = ""


while not findEnd(board, add): 
    add = nums.get()
    #print(add)
    for j in ["L", "R", "U", "D"]:
        put = add + j
        if valid(board, put):
            nums.put(put)

In [12]:
maze = []
maze.append(["#","#", "#", "#", "#", "O", "#", "#", "#"])
maze.append(["#"," ", " ", " ", " ", " ", " ", " ", "#"])
maze.append(["#"," ", "#", "#", " ", "#", "#", " ", "#"])
maze.append(["#"," ", "#", " ", " ", " ", "#", " ", "#"])
maze.append(["#"," ", "#", " ", "#", " ", "#", " ", "#"])
maze.append(["#"," ", "#", " ", "#", " ", "#", " ", "#"])
maze.append(["#"," ", "#", " ", "#", " ", "#", "#", "#"])
maze.append(["#"," ", " ", " ", " ", " ", " ", " ", "#"])
maze.append(["#","#", "#", "#", "#", "#", "#", "X", "#"])

In [13]:
maze

[['#', '#', '#', '#', '#', 'O', '#', '#', '#'],
 ['#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#'],
 ['#', ' ', '#', '#', ' ', '#', '#', ' ', '#'],
 ['#', ' ', '#', ' ', ' ', ' ', '#', ' ', '#'],
 ['#', ' ', '#', ' ', '#', ' ', '#', ' ', '#'],
 ['#', ' ', '#', ' ', '#', ' ', '#', ' ', '#'],
 ['#', ' ', '#', ' ', '#', ' ', '#', '#', '#'],
 ['#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#'],
 ['#', '#', '#', '#', '#', '#', '#', 'X', '#']]

In [ ]:
class Node:

    # Initialize the class
    def __init__(self, position:(), parent:()):
        self.position = position
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.position == other.position

    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f

    # Print node
    def __repr__(self):
        return ('({0},{1})'.format(self.position, self.f))
    
    

FileNotFoundError: [Errno 2] No such file or directory: 'maze.in'

In [4]:
int(math.sqrt((2)**2 + (4-0)**2)*10)

44

In [1]:
import math

In [3]:
def valid():
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            if (0 < i <20) and (0 < j <20):
                print("2")

In [4]:
valid()

2


In [1]:
from queue import PriorityQueue

q = PriorityQueue()


In [3]:
q.put((2,'ead'))

In [ ]:
x = q.get()

In [ ]:
while q.not_empty:
    print(q.get())

(2, 'ead')
(2, 'ead')


In [16]:
q.queue

[(2, 'ead')]

In [ ]:
x

In [ ]:
q.

In [5]:
def finished(obj, pos):
    obj[pos[0]][pos[1]].red = True
    if obj[pos[0]][pos[1]].old == (0,0):
        return
    else:
        finised(obj, obj[pos[0]][pos[1]].old)

In [22]:
min(max(20,10), 30, 20, 10)

10

In [30]:
(math.sqrt(1**2 + 1**2))*10

14.142135623730951